In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../Topic_modelling/us-consumer-finance-complaints"))

['database.sqlite', 'consumer_complaints.csv']


In [4]:
data = pd.read_csv('../Topic_modelling/us-consumer-finance-complaints/consumer_complaints.csv')
data.head(2)

/Users/patsnap/anaconda2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080


In [5]:
verbatim_product = data[['consumer_complaint_narrative','product']]
filtered_verbatim = verbatim_product.dropna()
filtered_verbatim.head(2)

,consumer_complaint_narrative,product
190126,XXXX has claimed I owe them {$27.00} for XXXX ...,Debt collection
190135,Due to inconsistencies in the amount owed that...,Consumer Loan


In [6]:
len(filtered_verbatim.consumer_complaint_narrative)

66806

In [7]:
filtered_verbatim['product'].value_counts()

Debt collection            17552
Mortgage                   14919
Credit reporting           12526
Credit card                 7929
Bank account or service     5711
Consumer Loan               3678
Student loan                2128
Prepaid card                 861
Payday loan                  726
Money transfers              666
Other financial service      110
Name: product, dtype: int64

In [8]:
filtered_verbatim['product'].value_counts().plot(kind='bar')

In [9]:
complaint = filtered_verbatim.iloc[1]['consumer_complaint_narrative']
pd.options.display.max_colwidth = 1000
print(complaint)

Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX XXXX 

In [10]:
import spacy #for our NLP processing
import nltk #to use the stopwords library
import string # for a list of all punctuation
from nltk.corpus import stopwords # for a list of stopwords

In [11]:
nlp = spacy.load('en_core_web_sm')
text = nlp(complaint)

In [12]:
text

Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX XXXX 

In [13]:
tokens = [tok.lemma_ for tok in text]
tokens = [tok.lemma_.lower().strip() for tok in text]
tokens = [tok.lemma_.lower().strip() for tok in text if tok.lemma_ != '-PRON-']

In [14]:
stop_words = stopwords.words('english')
punctuations = string.punctuation

In [15]:
tokens = [tok for tok in tokens if tok not in stop_words and tok not in punctuations]

In [16]:
def cleanup_text(complaint):
    doc = nlp(complaint, disable=['parser', 'ner'])
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in stop_words and tok not in punctuations]
    return tokens

In [17]:
limit = 100
doc_sample = filtered_verbatim.consumer_complaint_narrative
#print('tokenized and lemmatized document: ')

for idx, complaint in enumerate(doc_sample):
    #print(cleanup_text(complaint))
    if idx == limit:
        break

In [18]:
doc_sample = doc_sample[0:10000]

In [19]:
processed_docs = doc_sample.map(cleanup_text)

In [20]:
import gensim
dictionary = gensim.corpora.Dictionary(processed_docs)

In [21]:
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=100000)

In [22]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [23]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 15 ("pay") appears 2 time.
Word 18 ("send") appears 1 time.
Word 30 ("bank") appears 1 time.
Word 44 ("good") appears 1 time.
Word 69 ("report") appears 1 time.
Word 74 ("say") appears 2 time.
Word 94 ("account") appears 1 time.
Word 108 ("back") appears 1 time.
Word 115 ("come") appears 1 time.
Word 128 ("even") appears 1 time.
Word 134 ("finally") appears 1 time.
Word 165 ("money") appears 1 time.
Word 169 ("n't") appears 2 time.
Word 202 ("though") appears 1 time.
Word 203 ("time") appears 1 time.
Word 208 ("turn") appears 1 time.
Word 213 ("week") appears 1 time.
Word 218 ("would") appears 1 time.
Word 226 ("documentation") appears 1 time.
Word 229 ("open") appears 1 time.
Word 331 ("ask") appears 1 time.
Word 352 ("never") appears 1 time.
Word 360 ("refund") appears 2 time.
Word 379 ("another") appears 1 time.
Word 384 ("card") appears 4 time.
Word 388 ("charge") appears 4 time.
Word 463 ("different") appears 1 time.
Word 499 ("case") appears 2 time.
Word 562 ("dispute") appe

In [24]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2 )

In [25]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"credit" + 0.014*"account" + 0.013*"pay" + 0.013*"payment" + 0.013*"say" + 0.013*"would" + 0.012*"tell" + 0.011*"call" + 0.010*"get" + 0.009*"make"
Topic: 1 
Words: 0.029*"credit" + 0.022*"report" + 0.012*"call" + 0.012*"xx" + 0.011*"would" + 0.011*"receive" + 0.010*"account" + 0.009*"tell" + 0.009*"''" + 0.009*"send"
Topic: 2 
Words: 0.029*"credit" + 0.024*"account" + 0.022*"xx" + 0.021*"report" + 0.011*"payment" + 0.010*"collection" + 0.008*"debt" + 0.008*"call" + 0.008*"loan" + 0.008*"pay"
Topic: 3 
Words: 0.016*"mortgage" + 0.013*"request" + 0.012*"loan" + 0.011*"receive" + 0.008*"letter" + 0.007*"bank" + 0.007*"provide" + 0.006*"sale" + 0.006*"dispute" + 0.006*"home"
Topic: 4 
Words: 0.030*"credit" + 0.028*"report" + 0.023*"debt" + 0.016*"account" + 0.012*"pay" + 0.012*"state" + 0.010*"receive" + 0.010*"company" + 0.009*"information" + 0.009*"n't"
Topic: 5 
Words: 0.018*"account" + 0.013*"payment" + 0.009*"''" + 0.008*"bank" + 0.008*"file" + 0.008*"receive" 

In [28]:
#!pip install pyLDAvis

In [29]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis

In [38]:
vis_data = gensimvis.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(vis_data)

/Users/patsnap/anaconda2/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [36]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 10);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict)

In [37]:
num_topics = 8
get_lda_topics(lda_model, num_topics)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08
0,credit,credit,credit,mortgage,credit,account,call,account
1,account,report,account,request,report,payment,loan,card
2,pay,call,xx,loan,debt,'',debt,credit
3,payment,xx,report,receive,account,bank,n't,pay
4,say,would,payment,letter,pay,file,bank,charge
5,would,receive,collection,bank,state,receive,phone,tell
6,tell,account,debt,provide,receive,mortgage,wells,bank
7,call,tell,call,sale,company,make,number,would
8,get,'',loan,dispute,information,debt,would,receive
9,make,send,pay,home,n't,'s,fargo,call
